In [1]:
import numpy as np

import rpy2.robjects as robj
import rpy2.robjects.packages as rpackages
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

from rpy2.robjects.numpy2ri import numpy2ri

#define random seed
iter = 1000
np.random.seed(iter)

In [2]:
def pwr_calc(model, n, coef):
    #define noise
    if (model <= 16):
        noise = pow(10,(1/24*(coef-1)))
    
    #define models
    #model 1: linear with symmetric additive noise
    if (model == 1):
        xs = np.random.normal(size=n)
        ys = (2/3)*xs + noise * np.random.normal(n)
    #model 2: linear with asymmetric additive noise
    if (model == 2):
        xs = np.random.normal(size=n)
        ys = (0.05)*xs + noise * ((pow(np.random.normal(n),2)) - 1)
    #model 3: quadratic with additive noise
    if (model == 3):
        xs = np.random.normal(size=n)
        ys = (2/3)*xs*xs + noise * np.random.normal(n)

    #import jackknife approach library
    mJMI = rpackages.importr('mJMI')

    #reshape dataset to become a 2d array with 1 column
    xs = xs.reshape(-1,1)
    ys = ys.reshape(-1,1)
    
    #convert data to R matrix
    xr, xc = xs.shape
    x = robj.r.matrix(xs, nrow=xr, ncol=xc)
    yr, yc = ys.shape
    y = robj.r.matrix(ys, nrow=yr, ncol=yc)
  
    jmi = mJMI.mJMICpp(x,y,1000)
    
    print("The mutual information for model %s and coefficient %s is: %s" % (model, 
                                                                             coef, 
                                                                             jmi.rx2('mi')[0]))

In [3]:
model_length = 3
coef_ran = 3
n = 320

for model in range(1, model_length + 1):
    for coef in range(1, coef_ran + 1):
        pwr_calc(model, n, coef)

The mutual information for model 1 and coefficient 1 is: 5.202398342073842
The mutual information for model 1 and coefficient 2 is: 5.2023983420738436
The mutual information for model 1 and coefficient 3 is: 5.2023983420738436
The mutual information for model 2 and coefficient 1 is: 5.202398342073843
The mutual information for model 2 and coefficient 2 is: 5.202398342073841
The mutual information for model 2 and coefficient 3 is: 5.202398342073842
The mutual information for model 3 and coefficient 1 is: 4.693332156392547
The mutual information for model 3 and coefficient 2 is: 4.72418490208367
The mutual information for model 3 and coefficient 3 is: 4.67448146293298
